# जनरेटिव नेटवर्क्स

रीकरंट न्यूरल नेटवर्क्स (RNNs) आणि त्यांचे गेटेड सेल प्रकार जसे की लाँग शॉर्ट टर्म मेमरी सेल्स (LSTMs) आणि गेटेड रीकरंट युनिट्स (GRUs) यांनी भाषा मॉडेलिंगसाठी एक यंत्रणा प्रदान केली आहे, म्हणजेच ते शब्दांची क्रमवारी शिकू शकतात आणि अनुक्रमातील पुढील शब्दासाठी अंदाज देऊ शकतात. यामुळे RNNs चा वापर **जनरेटिव टास्क्स** साठी करता येतो, जसे की सामान्य मजकूर निर्मिती, मशीन अनुवाद, आणि अगदी प्रतिमेचे कॅप्शनिंग.

आम्ही मागील युनिटमध्ये चर्चा केलेल्या RNN आर्किटेक्चरमध्ये, प्रत्येक RNN युनिटने पुढील हिडन स्टेट आउटपुट म्हणून तयार केली. परंतु, आम्ही प्रत्येक रीकरंट युनिटला आणखी एक आउटपुट जोडू शकतो, ज्यामुळे आम्हाला एक **अनुक्रम** आउटपुट करता येईल (जो मूळ अनुक्रमाच्या लांबीइतकाच असेल). याशिवाय, आम्ही असे RNN युनिट्स वापरू शकतो जे प्रत्येक टप्प्यावर इनपुट स्वीकारत नाहीत, फक्त काही सुरुवातीचे स्टेट वेक्टर घेतात आणि नंतर आउटपुट्सचा अनुक्रम तयार करतात.

या नोटबुकमध्ये, आपण मजकूर तयार करण्यात मदत करणाऱ्या साध्या जनरेटिव मॉडेल्सवर लक्ष केंद्रित करू. सोप्या पद्धतीसाठी, आपण **कॅरेक्टर-लेव्हल नेटवर्क** तयार करूया, जे अक्षरानुसार मजकूर तयार करते. प्रशिक्षणादरम्यान, आपल्याला काही मजकूर कॉर्पस घ्यावा लागेल आणि त्याला अक्षरांच्या अनुक्रमांमध्ये विभागावे लागेल.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## अक्षर शब्दसंग्रह तयार करणे

अक्षर-स्तरीय जनरेटिव्ह नेटवर्क तयार करण्यासाठी, आपल्याला मजकूर शब्दांऐवजी स्वतंत्र अक्षरांमध्ये विभाजित करावा लागेल. `TextVectorization` लेयर ज्याचा आपण आधी वापर करत होतो, तो हे करू शकत नाही, त्यामुळे आपल्याकडे दोन पर्याय आहेत:

* मजकूर स्वतः लोड करा आणि 'हाताने' टोकनायझेशन करा, जसे [या अधिकृत Keras उदाहरणात](https://keras.io/examples/generative/lstm_character_level_text_generation/) दिले आहे.
* अक्षर-स्तरीय टोकनायझेशनसाठी `Tokenizer` वर्गाचा वापर करा.

आम्ही दुसरा पर्याय निवडणार आहोत. `Tokenizer` चा वापर शब्दांमध्ये टोकनायझेशनसाठी देखील केला जाऊ शकतो, त्यामुळे अक्षर-स्तरीय टोकनायझेशनपासून शब्द-स्तरीय टोकनायझेशनकडे सहजपणे स्विच करता येईल.

अक्षर-स्तरीय टोकनायझेशन करण्यासाठी, आपल्याला `char_level=True` हा पॅरामीटर पास करावा लागेल:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

आम्हाला **क्रमाचा शेवट** दर्शवण्यासाठी एक विशेष टोकन वापरायचे आहे, ज्याला आम्ही `<eos>` म्हणू. चला ते शब्दसंग्रहात हाताने जोडूया:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## शीर्षके तयार करण्यासाठी जनरेटिव्ह RNN प्रशिक्षण

आपण RNN कसे प्रशिक्षण देणार आहोत जेणेकरून ती बातम्यांची शीर्षके तयार करू शकेल, ते पुढीलप्रमाणे आहे. प्रत्येक टप्प्यावर, आपण एक शीर्षक घेऊ, जे RNN मध्ये दिले जाईल, आणि प्रत्येक इनपुट अक्षरासाठी आपण नेटवर्कला पुढील आउटपुट अक्षर तयार करण्यास सांगू:

!['HELLO' शब्दाच्या RNN जनरेशनचे उदाहरण दाखवणारी प्रतिमा.](../../../../../translated_images/mr/rnn-generate.56c54afb52f9781d.webp)

आमच्या अनुक्रमाच्या शेवटच्या अक्षरासाठी, आम्ही नेटवर्कला `<eos>` टोकन तयार करण्यास सांगू.

आपण येथे वापरत असलेल्या जनरेटिव्ह RNN आणि इतर RNN यामधील मुख्य फरक असा आहे की आपण RNN च्या प्रत्येक टप्प्याचा आउटपुट घेऊ, केवळ अंतिम सेलचा नाही. हे RNN सेलसाठी `return_sequences` पॅरामीटर निर्दिष्ट करून साध्य केले जाऊ शकते.

म्हणून, प्रशिक्षणादरम्यान, नेटवर्कसाठी इनपुट काही लांबीच्या एन्कोड केलेल्या अक्षरांचा अनुक्रम असेल, आणि आउटपुट देखील त्याच लांबीचा अनुक्रम असेल, परंतु एका घटकाने शिफ्ट केलेला आणि `<eos>` ने समाप्त केलेला असेल. मिनीबॅचमध्ये अशा अनेक अनुक्रमांचा समावेश असेल, आणि सर्व अनुक्रम एकसंध करण्यासाठी आपल्याला **padding** वापरण्याची गरज असेल.

चला, डेटासेटसाठी रूपांतर करणाऱ्या फंक्शन्स तयार करूया. कारण आपल्याला मिनीबॅच स्तरावर अनुक्रम padding करायचे आहे, आपण प्रथम `.batch()` कॉल करून डेटासेट बॅच करू, आणि नंतर `map` वापरून त्याचे रूपांतर करू. त्यामुळे, रूपांतरण फंक्शन संपूर्ण मिनीबॅचला पॅरामीटर म्हणून घेईल:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

आम्ही येथे काही महत्त्वाच्या गोष्टी करतो:
* सर्वप्रथम, आम्ही string tensor मधून वास्तविक मजकूर काढतो
* `text_to_sequences` मजकूरांच्या यादीला पूर्णांक tensor च्या यादीत रूपांतरित करते
* `pad_sequences` त्या tensor ना त्यांच्या जास्तीत जास्त लांबीपर्यंत padding करते
* शेवटी, आम्ही सर्व अक्षरांचे one-hot encode करतो, तसेच shifting आणि `<eos>` जोडणेही करतो. आम्हाला लवकरच कळेल की one-hot-encoded अक्षरे का आवश्यक आहेत

तथापि, ही फंक्शन **Pythonic** आहे, म्हणजेच ती Tensorflow computational graph मध्ये स्वयंचलितपणे रूपांतरित होऊ शकत नाही. जर आपण ही फंक्शन थेट `Dataset.map` फंक्शनमध्ये वापरण्याचा प्रयत्न केला, तर आपल्याला त्रुटी येतील. या Pythonic कॉलला `py_function` wrapper चा वापर करून enclosed करणे आवश्यक आहे:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **टीप**: Pythonic आणि Tensorflow ट्रान्सफॉर्मेशन फंक्शन्समधील फरक ओळखणे थोडेसे गुंतागुंतीचे वाटू शकते, आणि तुम्हाला प्रश्न पडू शकतो की, डेटासेटला `fit` मध्ये पास करण्यापूर्वी आपण ते स्टँडर्ड Python फंक्शन्स वापरून ट्रान्सफॉर्म का करत नाही. हे नक्कीच शक्य आहे, परंतु `Dataset.map` वापरण्याचा एक मोठा फायदा आहे, कारण डेटा ट्रान्सफॉर्मेशन पाईपलाइन Tensorflow च्या कम्प्युटेशनल ग्राफचा वापर करून कार्यान्वित होते, ज्यामुळे GPU च्या गणनात्मक क्षमतेचा फायदा घेतला जातो आणि CPU/GPU दरम्यान डेटा पास करण्याची गरज कमी होते.

आता आपण आपले जनरेटर नेटवर्क तयार करू शकतो आणि प्रशिक्षण सुरू करू शकतो. हे कोणत्याही पुनरावृत्ती सेलवर आधारित असू शकते, जे आपण मागील युनिटमध्ये (साधे, LSTM किंवा GRU) चर्चा केले होते. आपल्या उदाहरणात आपण LSTM वापरणार आहोत.

कारण नेटवर्क अक्षरे इनपुट म्हणून घेतो, आणि शब्दसंग्रहाचा आकार खूप लहान आहे, त्यामुळे आपल्याला एम्बेडिंग लेयरची गरज नाही; वन-हॉट-एनकोडेड इनपुट थेट LSTM सेलमध्ये जाऊ शकतो. आउटपुट लेयर `Dense` वर्गीकरणकर्ता असेल, जो LSTM आउटपुटला वन-हॉट-एनकोडेड टोकन क्रमांकांमध्ये रूपांतरित करेल.

याशिवाय, कारण आपण व्हेरिएबल-लांबीच्या सिक्वेन्सेसशी व्यवहार करत आहोत, आपण `Masking` लेयर वापरून एक मास्क तयार करू शकतो, जो स्ट्रिंगच्या पॅड केलेल्या भागाकडे दुर्लक्ष करेल. हे कठोरपणे आवश्यक नाही, कारण `<eos>` टोकनच्या पलीकडे असलेल्या गोष्टींमध्ये आपल्याला फारसा रस नाही, परंतु या प्रकारच्या लेयरसह काही अनुभव मिळवण्यासाठी आपण ते वापरणार आहोत. `input_shape` `(None, vocab_size)` असेल, जिथे `None` व्हेरिएबल लांबीच्या सिक्वेन्सेस दर्शवते, आणि आउटपुट आकार `(None, vocab_size)` असेल, जसे तुम्ही `summary` मधून पाहू शकता:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## आउटपुट तयार करणे

आता आपण मॉडेल प्रशिक्षण पूर्ण केले आहे, त्यामुळे आपण त्याचा वापर करून काही आउटपुट तयार करू इच्छितो. सर्वप्रथम, आपल्याला टोकन क्रमांकांच्या अनुक्रमाद्वारे दर्शविलेल्या मजकुराचे डिकोडिंग करण्याचा मार्ग आवश्यक आहे. यासाठी, आपण `tokenizer.sequences_to_texts` फंक्शन वापरू शकतो; परंतु, हे अक्षर-स्तरीय टोकनायझेशनसाठी चांगले कार्य करत नाही. त्यामुळे आपण टोकनायझरमधून (`word_index` नावाच्या) टोकन्सचा डिक्शनरी घेऊ, एक रिव्हर्स मॅप तयार करू, आणि आपले स्वतःचे डिकोडिंग फंक्शन लिहू:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

आता, आपण जनरेशन सुरू करूया. आपण काही स्ट्रिंग `start` ने सुरूवात करू, तिला `inp` या अनुक्रमामध्ये एन्कोड करू, आणि मग प्रत्येक टप्प्यावर आपले नेटवर्क कॉल करून पुढील अक्षर शोधू.

नेटवर्कचे आउटपुट `out` हे `vocab_size` घटकांचे एक वेक्टर असते, जे प्रत्येक टोकनच्या संभाव्यतेचे प्रतिनिधित्व करते. आपण `argmax` वापरून सर्वात संभाव्य टोकन क्रमांक शोधू शकतो. त्यानंतर आपण हे अक्षर तयार केलेल्या टोकन्सच्या यादीत जोडतो आणि जनरेशन पुढे सुरू ठेवतो. एका अक्षराची निर्मिती करण्याची ही प्रक्रिया `size` वेळा पुनरावृत्त केली जाते, जेणेकरून आवश्यक अक्षरांची संख्या तयार होईल. जर `eos_token` आढळला, तर आपण प्रक्रिया लवकर थांबवतो.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## प्रशिक्षणादरम्यान नमुना आउटपुट 

आपल्याकडे *अचूकता* यासारख्या उपयुक्त मेट्रिक्स उपलब्ध नसल्यामुळे, आपला मॉडेल सुधारत आहे का हे पाहण्याचा एकमेव मार्ग म्हणजे प्रशिक्षणादरम्यान तयार केलेल्या स्ट्रिंगचा **नमुना** घेणे. हे करण्यासाठी, आपण **कॉलबॅक्स** वापरणार आहोत, म्हणजेच अशा फंक्शन्स ज्या आपण `fit` फंक्शनला पास करू शकतो आणि त्या प्रशिक्षणादरम्यान ठराविक वेळांनी कॉल केल्या जातील.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


हे उदाहरण आधीच चांगला मजकूर तयार करते, पण काही प्रकारांनी यामध्ये आणखी सुधारणा करता येऊ शकते:
* **अधिक मजकूर**. आपण आपल्या कार्यासाठी फक्त शीर्षके वापरली आहेत, पण तुम्ही पूर्ण मजकूरावर प्रयोग करू शकता. लक्षात ठेवा की RNNs लांब अनुक्रम हाताळण्यात फारसे चांगले नसतात, त्यामुळे त्यांना लहान वाक्यांमध्ये विभागणे किंवा पूर्वनिर्धारित `num_chars` (उदा. 256) मूल्याच्या निश्चित अनुक्रम लांबीवर नेहमी प्रशिक्षण देणे योग्य ठरते. तुम्ही वरील उदाहरण अशा प्रकारच्या आर्किटेक्चरमध्ये बदलण्याचा प्रयत्न करू शकता, [अधिकृत Keras ट्यूटोरियल](https://keras.io/examples/generative/lstm_character_level_text_generation/) याचा प्रेरणादायक म्हणून वापर करून.
* **मल्टीलेयर LSTM**. LSTM सेल्सच्या 2 किंवा 3 स्तरांचा प्रयत्न करणे योग्य ठरते. जसे आपण मागील युनिटमध्ये उल्लेख केले होते, LSTM चा प्रत्येक स्तर मजकुरातून विशिष्ट नमुने काढतो, आणि कॅरेक्टर-लेव्हल जनरेटरच्या बाबतीत आपण अपेक्षा करू शकतो की खालचा LSTM स्तर अक्षरांचे संयोजन काढेल, आणि वरचे स्तर शब्द आणि शब्दांचे संयोजन काढतील. हे LSTM कन्स्ट्रक्टरला स्तरांची संख्या-पॅरामीटर पास करून सोप्या पद्धतीने अंमलात आणता येते.
* तुम्ही **GRU युनिट्स** वापरून पाहू शकता आणि कोणते चांगले कार्य करतात हे पाहू शकता, तसेच **विविध लपवलेल्या स्तरांचे आकार** वापरून प्रयोग करू शकता. खूप मोठा लपवलेला स्तर ओव्हरफिटिंग होऊ शकतो (उदा. नेटवर्क अचूक मजकूर शिकेल), आणि लहान आकार चांगले परिणाम देऊ शकत नाही.


## मऊ मजकूर निर्मिती आणि तापमान

`generate` च्या मागील व्याख्येमध्ये, आम्ही नेहमी उच्चतम संभाव्यतेचा वर्ण पुढील वर्ण म्हणून घेत होतो. यामुळे तयार केलेल्या मजकूरात वारंवार समान वर्ण अनुक्रम पुन्हा-पुन्हा येत होते, जसे या उदाहरणात:
```
today of the second the company and a second the company ...
```

तथापि, जर आपण पुढील वर्णासाठी संभाव्यता वितरण पाहिले, तर असे होऊ शकते की काही उच्चतम संभाव्यतांमधील फरक फारसा मोठा नसतो, उदा. एका वर्णाची संभाव्यता 0.2 असू शकते, तर दुसऱ्याची 0.19, इत्यादी. उदाहरणार्थ, जर आपण '*play*' अनुक्रमासाठी पुढील वर्ण शोधत असू, तर पुढील वर्ण जागा किंवा **e** (जसे *player* या शब्दात) असू शकतो.

यावरून असे निष्कर्ष काढता येते की उच्च संभाव्यतेचा वर्ण निवडणे नेहमी "योग्य" नसते, कारण दुसऱ्या क्रमांकाच्या उच्च संभाव्यतेचा वर्ण निवडल्यानेही अर्थपूर्ण मजकूर तयार होऊ शकतो. नेटवर्कने दिलेल्या संभाव्यता वितरणातून वर्ण **नमुन्याने** निवडणे अधिक शहाणपणाचे आहे.

हे नमुना निवडणे `np.multinomial` फंक्शन वापरून करता येते, जे तथाकथित **मल्टिनॉमियल वितरण** अंमलात आणते. खाली दिलेली फंक्शन ही **मऊ** मजकूर निर्मिती अंमलात आणते:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

आम्ही **temperature** नावाचा आणखी एक पॅरामीटर सादर केला आहे, जो आपण उच्चतम संभाव्यतेला किती कठोरपणे चिकटावे हे दर्शवण्यासाठी वापरला जातो. जर temperature 1.0 असेल, तर आम्ही योग्य बहुपद नमुना घेतो, आणि जेव्हा temperature अनंताकडे जाते - तेव्हा सर्व संभाव्यता समान होतात, आणि आम्ही पुढील अक्षर यादृच्छिकपणे निवडतो. खालील उदाहरणात आपण पाहू शकतो की जेव्हा आपण temperature खूप जास्त वाढवतो तेव्हा मजकूर अर्थहीन होतो, आणि जेव्हा तो 0 च्या जवळ जातो तेव्हा तो "सायकल केलेला" कठोर-निर्मित मजकूरासारखा दिसतो.



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
